In [1]:
using ParallelTemperingMonteCarlo
# using Pkg;Pkg.add("BenchmarkTools")
using BenchmarkTools
using Random,StaticArrays

# Equilibration

The purpose of this notebook is to write functions aimed at equilibrating the ptmc run. It must iterate over every trajectory for, by default 20\% of the total cycles run with _no_ sampling, other than testing the $\geq$ condition to update the min/max energies

In [2]:
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details

mc_cycles = 1000000 #default 20% equilibration cycles on top

mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)


Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

first, the $\geq$ condition: test whether it's faster to unpack mc_states or not. 

In [3]:
function testmin(new_en,en_min)
    if new_en < en_min
        en_min = new_en
    end
    return en_min
end

function test_min(new_en,en_min)
    return ifelse(new_en<en_min,new_en,en_min)
end
function test_max(new_en,en_max)
    return ifelse(new_en>en_max,new_en,en_max)
end
@benchmark testmin($rand(),$rand())
@benchmark test_min($rand(),$rand())

test_max (generic function with 1 method)

Use the ifelse version, it's slightly faster more often. NB the function guarantees type stability, telling the compiler what is to be returned, this is better.

In [6]:
function check_e_bounds(energy,ebounds::Vector)
    ebounds[1] = test_min(energy,ebounds[1])
    ebounds[2] = test_max(energy,ebounds[2])
    return ebounds
end
function checkbounds2(energy,emin,emax)
    emin = test_min(energy,emin)
    emax = test_max(energy,emax)
    return emin,emax
end
function checkbounds3(energy,ebounds)
    if energy<ebounds[1]
        ebounds[1]=energy
    elseif energy>ebounds[2]
        ebounds[2] = energy
    else
    end
    return ebounds
end
function checkbounds4(energy,ebounds)
    ebounds[1],ebounds[2] = test_min(energy,ebounds[1]),test_max(energy,ebounds[2])
    return ebounds 
end
ebounds = [-1.,1.]
energy = 0.6
Random.seed!(1234)
ebounds = check_e_bounds(energy,ebounds)
@benchmark check_e_bounds($rand(-1.5:1.5),$ebounds)
ebounds = [-1.,1.]
Random.seed!(1234)
@benchmark checkbounds2($rand(-1.5:1.5),$ebounds[1],$ebounds[2])
ebounds = [-1.,1.]
Random.seed!(1234)
energy = 0.6
@benchmark checkbounds3($rand(-1.5:1.5),$ebounds)
ebounds = [-1.,1.]
Random.seed!(1234)
energy = 0.6
ebounds = checkbounds4(energy,ebounds)
@benchmark checkbounds4($rand(-1.5:1.5),$ebounds)
import ParallelTemperingMonteCarlo.MCRun.mc_cycle!,ParallelTemperingMonteCarlo.MCRun.mc_step!,ParallelTemperingMonteCarlo.MCRun.update_max_stepsize!


checkbounds4 (generic function with 1 method)

In [ ]:

function test_equilibration(mc_states,move_strat,mc_params,pot,ensemble)#more booleans etc in existing code. Will include after testing


    a,v,r = atom_move_frequency(move_strat),vol_move_frequency(move_strat),rot_move_frequency(move_strat)
    n_steps = a + v + r
    println("Total number of moves per MC cycle: ", n_steps)
    println()

    ebounds = [100. , -100.]

    for i = 1:mc_params.eq_cycles
        mc_states = mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,n_steps,a,v,r)
        
        for state in mc_states
            ebounds = checkbounds3(state.en_tot,ebounds)
        end

        if rem(i, mc_params.n_adjust) == 0
            for i_traj = 1:mc_params.n_traj
                update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
            end 
        end
    end
    return mc_states,ebounds,a,v,r
end
@benchmark test_equilibration($mc_states,move_strat,mc_params,pot,ensemble)

NB: Tested the three "sensible" methods for checking the energy and the fastest overall was iteration checkbounds3, likely as a result of it not calculating max/min if the other is satisfied first. The major quesiton is which goes first? This could have a huge impact on how quick the process is: Testing time:

In [13]:
function checkbounds3a(energy,ebounds)
    #if energy<ebounds[1]
        # ebounds[1]=energy
    if energy>ebounds[2]
        ebounds[2] = energy
    elseif energy<ebounds[1]
        ebounds[1]=energy
    else
    end
    return ebounds
end
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

function test_equilibration2(mc_states,move_strat,mc_params,pot,ensemble)#more booleans etc in existing code. Will include after testing


    a,v,r = atom_move_frequency(move_strat),vol_move_frequency(move_strat),rot_move_frequency(move_strat)
    n_steps = a + v + r
    println("Total number of moves per MC cycle: ", n_steps)
    println()

    ebounds = [100. , -100.]

    for i = 1:mc_params.eq_cycles
        mc_states = mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,n_steps,a,v,r)
        
        for state in mc_states
            ebounds = checkbounds3a(state.en_tot,ebounds)
        end

        if rem(i, mc_params.n_adjust) == 0
            for i_traj = 1:mc_params.n_traj
                update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
            end 
        end
    end
    return mc_states,ebounds,a,v,r
end
@benchmark test_equilibration2($mc_states,move_strat,mc_params,pot,ensemble)

checkbounds3a (generic function with 1 method)

# Equilibration complete



In [16]:
function reset_counters(state)
    state.count_atom = [0,0]
    state.count_vol = [0,0]
    state.count_rot = [0,0]
    state.count_exc = [0,0]
end

function test_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)#more booleans etc in existing code. Will include after testing


    a,v,r = atom_move_frequency(move_strat),vol_move_frequency(move_strat),rot_move_frequency(move_strat)
    n_steps = a + v + r
    println("Total number of moves per MC cycle: ", n_steps)
    println()

    ebounds = [100. , -100.]

    for i = 1:mc_params.eq_cycles

        mc_states = mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,n_steps,a,v,r)
        
        for state in mc_states
            ebounds = checkbounds3(state.en_tot,ebounds)
        end

        if rem(i, mc_params.n_adjust) == 0
            for state in mc_states
                update_max_stepsize!(state,mc_params.n_adjust,a,v,r)
            end
        end
        
    end

    #reset counter-variables
    for state in mc_states
        reset_counters(state)
    end

    delta_en_hist,delta_r2 = initialise_histograms!(mc_params,results,ebounds,mc_states[1].config.bc)

    start_counter = 1

    println("equilibration done")

    return mc_states,results,delta_en_hist,delta_r2,start_counter,n_steps,a,v,r

end
function bench_test1(mc_states,mc_params)
    for i_traj = 1:mc_params.n_traj
        reset_counters(mc_states[i_traj])
    end
    return mc_states
end
function bench_test2(mc_states)
    for state in mc_states
        reset_counters(state)
    end
    return mc_states
end
@benchmark bench_test1(mc_states,mc_params)
@benchmark bench_test2(mc_states)

test_equilibration (generic function with 2 methods)

It seems for state in states is better than iterating 1:mc_cycles

In [18]:
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]
Random.seed!(1234)

mc_states,results,delta_en_hist,delta_r2,start_counter,a,v,r = test_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)

save_states(mc_params,mc_states,0,pwd())

Total number of moves per MC cycle: 13



equilibration done


In [19]:
function test_ptmc_cycle!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham::Bool = false, save::Bool=false, restart::Bool=false,restartindex::Int64=0,save_dir = pwd())



    
    
    
    #If we do not save the hamiltonian we still need to store the E, E**2 terms at each cycle
    for i_traj = 1:mc_params.n_traj
        push!(mc_states[i_traj].ham, 0)
        push!(mc_states[i_traj].ham, 0)
    end
            
    mc_states,results,delta_en_hist,delta_r2,start_counter,n_steps,a,v,r = test_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)
        
        
                #if save == true
    save_states(mc_params,mc_states,0,save_dir)
                #end
            #end
        
        
        
            #main MC loop
        
            #if restart == false
        
    for i = 1:mc_params.mc_cycles
        @inbounds ptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r,save,i,save_dir,delta_en_hist,delta_r2)
    end
        
            # else #if restarting
        
            #     for i = restartindex:mc_params.mc_cycles
            #         @inbounds ptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r,save,i,save_dir,delta_en_hist,delta_r2)
            #     end 
            # end
    println("MC loop done.")
            #Evaluation
            #average energy
            n_sample = mc_params.mc_cycles / mc_params.mc_sample
        
            if save_ham == true
                en_avg = [sum(mc_states[i_traj].ham) / n_sample for i_traj in 1:mc_params.n_traj] #floor(mc_cycles/mc_sample)
                en2_avg = [sum(mc_states[i_traj].ham .* mc_states[i_traj].ham) / n_sample for i_traj in 1:mc_params.n_traj]
            else
                en_avg = [mc_states[i_traj].ham[1] / n_sample  for i_traj in 1:mc_params.n_traj]
                en2_avg = [mc_states[i_traj].ham[2] / n_sample  for i_traj in 1:mc_params.n_traj]
            end
        
        
            results.en_avg = en_avg
        
            #heat capacity
            results.heat_cap = [(en2_avg[i]-en_avg[i]^2) * mc_states[i].beta 
            
            for i in 1:mc_params.n_traj]
        
            #acceptance statistics
            results.count_stat_atom = [mc_states[i_traj].count_atom[1] / (mc_params.n_atoms * mc_params.mc_cycles) for i_traj in 1:mc_params.n_traj]
            results.count_stat_exc = [mc_states[i_traj].count_exc[2] / mc_states[i_traj].count_exc[1] for i_traj in 1:mc_params.n_traj]
        
            println(results.heat_cap)
        
            #energy histograms
        
            #TO DO
            # volume (NPT ensemble),rot moves ...
            # move boundary condition from config to mc_params?
            # rdfs
        
            println("done")
            return 
end


test_ptmc_cycle! (generic function with 1 method)

In [20]:
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]
Random.seed!(1234)

@time test_ptmc_cycle!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=false)


Total number of moves per MC cycle: 13



equilibration done


MC loop done.


[0.0003234110423948385, 0.00034130435146749016, 0.0003571979730485569, 0.0003751826872688714, 0.00038929116332445156, 0.00041586364789964576, 0.00043669694264594094, 0.0004595895593319723, 0.0004890435704389878, 0.0005157634753972615, 0.0005536751008066922, 0.0005723147050125483, 0.0006169481887285358, 0.000683777774995162, 0.0007252243017238607, 0.000822837124956102, 0.0008936290651810269, 0.0009485259670130599, 0.0011550216114052253, 0.001486674414643203, 0.0017357931127196526, 0.0022538799773156406, 0.002685202167626738, 0.0030124683418126814, 0.003227185612361041, 0.0031974041582545276, 0.003076489630752461, 0.002817309162204895, 0.0025391775980580186, 0.0022260287657100224, 0.002074304958103143, 0.0020101649064780125]
done
158.312354 seconds (1.08 G allocations: 184.391 GiB, 9.34% gc time, 0.19% compilation time)


In [22]:
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)
Random.seed!(1234)
@time ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=false)


Total number of moves per MC cycle: 13



equilibration done


MC loop done.
[0.0003234110423948385, 0.00034130435146749016, 0.0003571979730485569, 0.0003751826872688714, 0.00038929116332445156, 0.00041586364789964576, 0.00043669694264594094, 0.0004595895593319723, 0.0004890435704389878, 0.0005157634753972615, 0.0005536751008066922, 0.0005723147050125483, 0.0006169481887285358, 0.000683777774995162, 0.0007252243017238607, 0.000822837124956102, 0.0008936290651810269, 0.0009485259670130599, 0.0011550216114052253, 0.001486674414643203, 0.0017357931127196526, 0.0022538799773156406, 0.002685202167626738, 0.0030124683418126814, 0.003227185612361041, 0.0031974041582545276, 0.003076489630752461, 0.002817309162204895, 0.0025391775980580186, 0.0022260287657100224, 0.002074304958103143, 0.0020101649064780125]
done
159.323036 seconds (1.11 G allocations: 184.952 GiB, 9.38% gc time, 0.15% compilation time)
